In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np

In [2]:
DATA_PATH = '../../data/combined_tripdata.csv'
data = pd.read_csv(DATA_PATH)
print(data.head())

C:\Users\swapn\AppData\Local\Temp\ipykernel_19564\1721040238.py:2: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(DATA_PATH)


            ride_id rideable_type           started_at             ended_at  \
0  A847FADBBC638E45   docked_bike  2020-04-26 17:45:14  2020-04-26 18:12:03   
1  5405B80E996FF60D   docked_bike  2020-04-17 17:08:54  2020-04-17 17:17:03   
2  5DD24A79A4E006F4   docked_bike  2020-04-01 17:54:13  2020-04-01 18:08:36   
3  2A59BBDF5CDBA725   docked_bike  2020-04-07 12:50:19  2020-04-07 13:02:31   
4  27AD306C119C6158   docked_bike  2020-04-18 10:22:59  2020-04-18 11:15:54   

             start_station_name start_station_id             end_station_name  \
0                  Eckhart Park             86.0  Lincoln Ave & Diversey Pkwy   
1     Drake Ave & Fullerton Ave            503.0              Kosciuszko Park   
2          McClurg Ct & Erie St            142.0   Indiana Ave & Roosevelt Rd   
3  California Ave & Division St            216.0       Wood St & Augusta Blvd   
4          Rush St & Hubbard St            125.0   Sheridan Rd & Lawrence Ave   

  end_station_id  start_lat  start_lng

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25256682 entries, 0 to 25256681
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 2.4+ GB
None


In [4]:
print(data.describe())

          start_lat     start_lng       end_lat       end_lng
count  2.525668e+07  2.525668e+07  2.522829e+07  2.522829e+07
mean   4.190235e+01 -8.764650e+01  4.190264e+01 -8.764668e+01
std    4.529851e-02  2.808681e-02  5.557035e-02  8.342695e-02
min    4.163000e+01 -8.794000e+01  0.000000e+00 -1.440500e+02
25%    4.188103e+01 -8.766000e+01  4.188132e+01 -8.766014e+01
50%    4.189897e+01 -8.764312e+01  4.189993e+01 -8.764331e+01
75%    4.193000e+01 -8.762932e+01  4.193000e+01 -8.762932e+01
max    4.563503e+01 -7.379648e+01  8.796000e+01  1.525300e+02


In [5]:
# Check for non-standard datetime formats in the 'started_at' column
data['started_at'] = data['started_at'].astype(str)  # Ensure it's a string for processing
data['ended_at'] = data['ended_at'].astype(str)

In [6]:
# Identify problematic rows
invalid_start_times = data[~data['started_at'].str.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}(\.\d+)?', na=False)]
invalid_end_times = data[~data['ended_at'].str.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}(\.\d+)?', na=False)]
# Display problematic rows
invalid_start_times.head(), invalid_end_times.head()

(Empty DataFrame
 Columns: [ride_id, rideable_type, started_at, ended_at, start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat, end_lng, member_casual]
 Index: [],
 Empty DataFrame
 Columns: [ride_id, rideable_type, started_at, ended_at, start_station_name, start_station_id, end_station_name, end_station_id, start_lat, start_lng, end_lat, end_lng, member_casual]
 Index: [])

In [7]:
# Convert timestamps while handling extra milliseconds
data['started_at'] = pd.to_datetime(data['started_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
data['ended_at'] = pd.to_datetime(data['ended_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')


In [8]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25256682 entries, 0 to 25256681
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 2.4+ GB
None


In [9]:
# Drop rows with invalid datetime conversion (NaT values)
data = data.dropna(subset=['started_at', 'ended_at'])

# Verify the conversion
data[['started_at', 'ended_at']].head()

,started_at,ended_at
0,2020-04-26 17:45:14,2020-04-26 18:12:03
1,2020-04-17 17:08:54,2020-04-17 17:17:03
2,2020-04-01 17:54:13,2020-04-01 18:08:36
3,2020-04-07 12:50:19,2020-04-07 13:02:31
4,2020-04-18 10:22:59,2020-04-18 11:15:54


In [10]:
# Remove rows where 'ended_at' is earlier than 'started_at' (data inconsistencies)
data = data[data['ended_at'] >= data['started_at']]

In [11]:
# Compute ride duration in minutes
# Compute ride distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of Earth in km
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

data['ride_duration'] = (data['ended_at'] - data['started_at']).dt.total_seconds() / 60
data['ride_distance_km'] = data.apply(lambda row: haversine(row['start_lat'], row['start_lng'], row['end_lat'], row['end_lng']), axis=1)
data['start_hour'] = data['started_at'].dt.hour
data['start_dayofweek'] = data['started_at'].dt.dayofweek


In [12]:
print(data.describe())

                          started_at                       ended_at  \
count                       21079114                       21079114   
mean   2022-05-30 07:51:03.722923264  2022-05-30 08:12:00.431092736   
min              2020-04-01 00:00:30            2020-04-01 00:10:45   
25%    2021-07-08 05:37:20.750000128            2021-07-08 06:00:42   
50%       2022-06-15 06:33:41.500000     2022-06-15 06:53:23.500000   
75%    2023-06-20 19:03:28.750000128            2023-06-20 19:20:20   
max              2024-05-31 23:59:57            2024-06-02 00:56:55   
std                              NaN                            NaN   

          start_lat     start_lng       end_lat       end_lng  ride_duration  \
count  2.107911e+07  2.107911e+07  2.105571e+07  2.105571e+07   2.107911e+07   
mean   4.190228e+01 -8.764658e+01  4.190256e+01 -8.764675e+01   2.094514e+01   
min    4.163000e+01 -8.794000e+01  0.000000e+00 -8.897000e+01   0.000000e+00   
25%    4.188111e+01 -8.766000e+01  4.188

In [13]:
#convert the start_dayofweek to a string
data['start_dayofweek'] = data['started_at'].dt.dayofweek
data['start_dayofweek'] = data['start_dayofweek'].replace({0: 'Mon', 1: 'Tue', 2: 'Wed', 3: 'Thur', 4: 'Fri', 5: 'Sat', 6: 'Sun'})

In [14]:
#drop ride_id column
data = data.drop(columns=['ride_id'])   
data = data.drop(columns=['start_station_id'])
data = data.drop(columns=['end_station_id'])
data = data.drop(columns=['ended_at'])
data = data.drop(columns=['end_station_name']) 
data = data.drop(columns=['end_lat'])
data = data.drop(columns=['end_lng'])
data.head()

,rideable_type,started_at,start_station_name,start_lat,start_lng,member_casual,ride_duration,ride_distance_km,start_hour,start_dayofweek
0,docked_bike,2020-04-26 17:45:14,Eckhart Park,41.8964,-87.6610,member,26.816667,3.985729,17,Sun
1,docked_bike,2020-04-17 17:08:54,Drake Ave & Fullerton Ave,41.9244,-87.7154,member,8.150000,0.978873,17,Fri
2,docked_bike,2020-04-01 17:54:13,McClurg Ct & Erie St,41.8945,-87.6179,member,14.383333,2.987769,17,Wed
3,docked_bike,2020-04-07 12:50:19,California Ave & Division St,41.9030,-87.6975,member,12.200000,2.136093,12,Tue
4,docked_bike,2020-04-18 10:22:59,Rush St & Hubbard St,41.8902,-87.6262,casual,52.916667,9.127509,10,Sat


In [15]:
#print distinct rideable_type
print(data['rideable_type'].unique())
replace_map = {'rideable_type': {'docked_bike': 'Docked', 'electric_bike': 'Electric', 'classic_bike': 'Classic'}}
data.replace(replace_map, inplace=True)
#Replace member_casual values like 'member' with 'Member' and 'casual' with 'Casual'   
replace_map = {'member_casual': {'member': 'Member', 'casual': 'Casual'}}
data.replace(replace_map, inplace=True)

['docked_bike' 'electric_bike' 'classic_bike']


In [16]:
# print min and max started_at
print(data['started_at'].min())
print(data['started_at'].max())
print(data['started_at'].dt.year.value_counts())


data_2024 = data[data['started_at'].dt.year >= 2023]
data_2024.to_csv('../../data/2023_2024_tripdata.csv', index=False)

2020-04-01 00:00:30
2024-05-31 23:59:57
started_at
2023    5719605
2021    5594916
2022    4966287
2020    3104248
2024    1694058
Name: count, dtype: int64


In [17]:
# read file actual_vs_predicted.csv and print the first 5 rows
df = pd.read_csv('../../data/actual_vs_predicted.csv')
print(df.head())

   Unnamed: 0  Actual  Predictions           started_at
0    20205345    4.62     9.314082  2024-04-26 12:58:49
1    20205346    7.78     9.976063  2024-04-08 21:07:05
2    20205347    4.48    12.069504  2024-04-10 10:20:11
3    20205348    3.37    12.069504  2024-04-10 06:10:16
4    20205349    4.22    12.069504  2024-04-10 16:00:44


In [18]:
# filter and export the data with started_at for year 2023 onwards
df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df_2023 = df[df['started_at'].dt.year >= 2023]
df_2023.to_csv('../../data/2023_2024_actual_vs_predicted.csv', index=False)

